## The "with" statement (PEP 343)

runtime context를 다루는 방법입니다.
> Python’s with statement supports the concept of a <span class="mark">runtime context</span> defined by a context manager. This is implemented using a pair of methods that allow user-defined classes to define a runtime context that is entered before the statement body is executed and exited when the statement ends:

파이썬 공식 Reference 에서 발췌

일단 with문은 <span class="burk">try-finally 패턴을 단순화</span>할 수 있습니다. <i class="fa fa-lightbulb-o "></i>.
- finally는 보통 중요한 리소스를 해제하거나, 상태 복원을 위해서 사용합니다.

In [48]:
# 가장 간단한 예
try:
    fp = open('data/myfile.txt', 'rb')
    print(len(fp.read()))
except IOError:
    pass
finally:
    fp.close()

253


In [49]:
# 다음과 같이 바꿀 수 있습니다.
with open('data/myfile.txt', 'rb') as fp:
    print(len(fp.read()))
# fp is nothing.

253


In [53]:
# 여러개를 한번에 처리할 수 있어요.
with open('data/myfile.txt', 'rb') as fp1, \
     open('data/myfile copy.txt', 'rb') as fp2:
    data1, data2 = fp1.read(), fp2.read()
    print(len(data1), len(data2))

253 253


In [2]:
# 변수범위를 확인하세요.
with open('data/myfile.txt', 'rb') as fp1, \
     open('data/myfile copy.txt', 'rb') as fp2:
    data1, data2 = fp1.read(), fp2.read()

print(len(data1), len(data2))

'e'

<span class="burk">좋은소식!</span> Python의 **`__enter__`, `__exit__` 프로토콜**로 직접 구현할 수 있어요.
- 시작은 `__enter__`
- 끝은 `__exit__`

In [4]:
class MyFileOpener:
    def __enter__(self):
        print("opened.")
        self.fp = open('data/myfile.txt', 'rb')
        return self.fp
    def __exit__(self, exc_type, exc_value, traceback):
        self.fp.close()
        print("closed.")

with MyFileOpener() as f:
    print(len(f.read()))

opened.
253
closed.


Exception: e

In [42]:
class MyFileOpener:
    def __enter__(self):
        self.fp = open('data/myfile.txt', 'rb')
        return self.fp
    def __exit__(self, exc_type, exc_value, traceback):
        print('__exit__ start', exc_type, exc_value, traceback)
        self.fp.close()
        print('__exit__ end')

# 예외는 바깥으로 나가요.
with MyFileOpener() as f:
    f.seek(-1)
    print(len(f.read()))

__exit__ start <class 'OSError'> [Errno 22] Invalid argument <traceback object at 0x10d68a848>
__exit__ end


OSError: [Errno 22] Invalid argument

## 다시 runtime context관점으로 돌아가보겠습니다.

In [6]:
class HoldingStdout:

    def __enter__(self):
        import sys
        import io
        self._write = sys.stdout.write
        self.string_io = io.StringIO()
        sys.stdout.write = self.string_io.write
        return 'dummy'

    def __exit__(self, exc_type, exc_value, traceback):
        import sys
        sys.stdout.write = self._write
        print(self.string_io.getvalue(), end='')

In [8]:
import time
with HoldingStdout() as _:
    print('HelloWorld:p')
    time.sleep(1)
print('End')

HelloWorld:p
End


with문 안에서만 다른 문맥(context)을 갖고 동작하도록 만들 수 있습니다.
- 많은 라이브러리들이 with문을 사용할 수 있도록 구현되어 있습니다.
- cx_Oracle, sqlite3, threading

```
>>> import cx_Oracle
>>> with cx_Oracle.connect('system/0009') as conn:
...     print(conn)
```

In [5]:
# with문이 구현안된 close를 사용하는 리소스 관리
from contextlib import contextmanager

@contextmanager
def closing(thing):
    try:
        yield thing
    finally:
        thing.close()

In [3]:
# with문이 구현안된 close를 사용하는 리소스 관리
from contextlib import closing
from urllib.request import urlopen

with closing(urlopen('http://www.python.org')) as page:
    for line in page:
        print(line)
        break

b'<!doctype html>\n'


In [16]:
import threading
lock = threading.Lock()

# old-way
lock.acquire()
try:
    print('Critical section')
finally:
    lock.release()

# new-way
with lock:
    print('Critical section')

Critical section
Critical section


## context를 구현하는 (좀 더 직관적인) 다른 방법이 있습니다.
*함수는 클래스보다 구현하기 쉽고, 직관적입니다.*

In [7]:
# 아까 봤던 가장 간단한 예
def myfile_reader():
    try:
        fp = open('data/myfile.txt', 'rb')
        # do something
    except IOError:
        pass
    finally:
        fp.close()

`fp` <span class="mark"><여기를 끊어주면></span> `open('data/myfile.txt', 'rb')`

In [11]:
# 아까 봤던 가장 간단한 예
def myfile_reader():
    try:
        fp = open('data/myfile.txt', 'rb')
        yield fp
    except IOError:
        pass
    finally:
        fp.close()
        
myfile_reader()

<generator object myfile_reader at 0x105ae24c0>

In [11]:
import contextlib 
# 아까 봤던 가장 간단한 예
@contextlib.contextmanager
def myfile_reader():
    try:
        fp = open('data/myfile.txt', 'rb')
        yield fp
    except IOError:
        pass
    finally:
        fp.close()

In [17]:
with myfile_reader() as f:
    print(len(f.read()))

253


In [10]:
# 재밌는 예
# https://docs.python.org/3.6/library/contextlib.html
from contextlib import contextmanager

@contextmanager
def tag(name):
    print("<%s>" % name)
    yield
    print("</%s>" % name)

with tag("h1"):
    print("foo")
    print("bar")

<h1>
foo
bar
</h1>


In [19]:
# 재밌는 예
# https://speakerdeck.com/pyconslides/transforming-code-into-beautiful-idiomatic-python-by-raymond-hettinger-1?slide=34
import os, contextlib
@contextlib.contextmanager
def ignored(*exceptions):
    try:
        yield
    except exceptions:
        pass

with ignored(OSError, IOError, NameError):
    os.remove('somefile.tmp')

우리가 코드를 refactoring할때
- A<span class="burk">B</span>C, D<span class="burk">B</span>F
 -> **B를 분리**

- <span class="burk">A</span>B<span class="burk">C</span>, <span class="burk">A</span>D<span class="burk">C</span>
 -> **A와 C를 분리**

두번째 케이스에서 쉽게 사용할 수 있습니다.

yield는 보통 반복문에서 사용하는 것으로 지금까지 설명했지만,
위와 같이 **흐름제어를 위해서도** 사용합니다. 이 과정에서는 다루지 않지만 coroutine과 같은 개념에서 다시 만나게 됩니다.

- with문 자체는 최초에는 리소스관리를 위해 등장했지만 (열고닫기)
- 공통적으로 사용되는 준비/마무리 과정을 구현하기 위해서도 사용됩니다.